# 🦟 MINT Workshop: Malaria Intervention Scenario Modeling

**Duration:** ~2 hours  
**Audience:** Researchers (primarily R programmers)  
**Goal:** Learn to use the `minte` package to explore malaria intervention scenarios

---

## What You'll Learn

1. Basic Python syntax (for R users)
2. Running single and multiple scenarios
3. Interpreting results (prevalence & cases)
4. Creating and customizing plots
5. Batch processing hundreds of scenarios
6. Exporting results for further analysis

---

## 📦 Part 1: Setup & Installation (~10 min)

Run this cell first - it installs the package and dependencies.

In [ ]:
# Install the minte package (run once per session)
# Replace with your actual package installation method
!pip install minte --quiet

# Or if installing from GitHub:
# !pip install git+https://github.com/YOUR_ORG/minte.git --quiet

# Or if installing from a local wheel:
# from google.colab import files
# uploaded = files.upload()  # Upload the .whl file
# !pip install minte-*.whl --quiet

print("✅ Installation complete!")

In [ ]:
# Import everything we need
from minte import run_minter_scenarios, create_scenario_plots
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# For downloading files later
from google.colab import files
import os

print("✅ All imports successful!")

---

## 🔤 Part 2: Python for R Users (~15 min)

If you're coming from R, here are the key differences:

### Quick Reference Table

| Concept | R | Python |
|---------|---|--------|
| Assignment | `x <- 5` or `x = 5` | `x = 5` |
| Vectors/Lists | `c(1, 2, 3)` | `[1, 2, 3]` |
| Indexing | `x[1]` (1-based) | `x[0]` (0-based!) |
| Range | `1:5` → 1,2,3,4,5 | `range(1,5)` → 1,2,3,4 |
| Comments | `# comment` | `# comment` |
| NULL/None | `NULL` | `None` |
| TRUE/FALSE | `TRUE` / `FALSE` | `True` / `False` |
| DataFrames | `df$column` | `df['column']` or `df.column` |
| Print | `print(x)` | `print(x)` |
| Functions | `function(x) {...}` | `def func(x):` |

In [ ]:
# 🎯 CRITICAL DIFFERENCE: Python uses 0-based indexing!

my_list = ["A", "B", "C", "D"]

# In R: my_list[1] would give "A"
# In Python:
print(f"First element: {my_list[0]}")   # A
print(f"Second element: {my_list[1]}")  # B
print(f"Last element: {my_list[-1]}")   # D (negative indexing = from end)

In [ ]:
# Lists vs Vectors
# In R you'd write: c(0.2, 0.2, 0.2)
# In Python, use square brackets:

resistance_levels = [0.2, 0.2, 0.2]
print(f"Resistance levels: {resistance_levels}")

# To repeat a value (like R's rep(0.2, 5)):
repeated = [0.2] * 5
print(f"Repeated 5 times: {repeated}")

In [ ]:
# Accessing DataFrame columns (like R's $ operator)

# Create a simple DataFrame
df = pd.DataFrame({
    'scenario': ['A', 'B', 'C'],
    'prevalence': [0.3, 0.4, 0.5]
})

# In R: df$prevalence
# In Python (two equivalent ways):
print(df['prevalence'])  # Bracket notation (always works)
print(df.prevalence)     # Dot notation (simpler but doesn't work with spaces)

In [ ]:
# None vs NULL
# In R you use NULL, in Python use None

net_type = None  # No specific net type

# Lists with None values:
net_types = [None, None, "py_pbo", "py_ppf"]
print(net_types)

### 💡 Pro Tip: Named Arguments

Just like R, Python supports named arguments. This makes your code much clearer!

In [ ]:
# Instead of relying on position:
# run_minter_scenarios([0.2], [0.3], [0.2], ...)  # Hard to read!

# Use named arguments (recommended):
# run_minter_scenarios(
#     res_use = [0.2],
#     py_only = [0.3],
#     py_pbo  = [0.2],
#     ...
# )

print("Named arguments make code readable and less error-prone!")

---

## 🎯 Part 3: Your First Scenario (~20 min)

Let's start simple with a single baseline scenario.

### Understanding the Parameters

| Parameter | Description | Typical Range |
|-----------|-------------|---------------|
| `res_use` | Current insecticide resistance level | 0.0 - 1.0 |
| `py_only` | Pyrethroid-only net coverage | 0.0 - 1.0 |
| `py_pbo` | Pyrethroid-PBO net coverage | 0.0 - 1.0 |
| `py_pyrrole` | Pyrethroid-Pyrrole net coverage | 0.0 - 1.0 |
| `py_ppf` | Pyrethroid-PPF (IG2) net coverage | 0.0 - 1.0 |
| `prev` | Baseline malaria prevalence | 0.0 - 1.0 |
| `Q0` | Proportion of bites taken indoors | 0.0 - 1.0 |
| `phi` | Phi bednet (proportion of bites at bedtime) | 0.0 - 1.0 |
| `season` | Seasonality (0 = non-seasonal, 1 = seasonal) | 0 or 1 |
| `routine` | Routine treatment available | 0 or 1 |
| `irs` | Current IRS (indoor residual spraying) coverage | 0.0 - 1.0 |
| `irs_future` | Future IRS coverage | 0.0 - 1.0 |
| `lsm` | Larval source management coverage | 0.0 - 1.0 |
| `itn_future` | Future ITN coverage (0-1) | 0.0 - 1.0 |
| `net_type_future` | Future net type | py_only, py_pbo, py_pyrrole, py_ppf, or None |

In [ ]:
# Run a single baseline scenario
# This represents a typical high-transmission area with no future intervention

baseline = run_minter_scenarios(
    # Scenario identification
    scenario_tag   = ["baseline"],
    
    # Current net distribution (sums can exceed 1 if nets overlap in time)
    res_use        = [0.2],   # 20% resistance level
    py_only        = [0.3],   # 30% pyrethroid-only nets
    py_pbo         = [0.2],   # 20% PBO nets
    py_pyrrole     = [0.1],   # 10% pyrrole nets
    py_ppf         = [0.05],  # 5% PPF (IG2) nets
    
    # Epidemiological setting
    prev           = [0.55],  # 55% baseline prevalence
    Q0             = [0.92],  # 92% indoor biting
    phi            = [0.85],  # 85% biting at bedtime
    season         = [0],     # Non-seasonal
    routine        = [0],     # No routine treatment
    
    # Current and future interventions
    irs            = [0.4],   # 40% current IRS
    irs_future     = [0.0],   # No future IRS
    lsm            = [0.0],   # No LSM
    itn_future     = [0.0],   # No future ITN distribution
    net_type_future= [None],  # No specific net type planned
)

print("✅ Scenario complete!")

In [ ]:
# Examine the results structure
print("Available outputs:")
print(baseline.keys())

print("\n--- Prevalence Results (first 10 rows) ---")
print(baseline.prevalence.head(10))

print("\n--- Cases Results (first 10 rows) ---")
print(baseline.cases.head(10))

In [ ]:
# Check if EIR predictions are valid
print(f"EIR Valid: {baseline.eir_valid}")
print(f"\nScenario Metadata:")
print(baseline.scenario_meta)

In [ ]:
# Quick plot of the baseline
os.makedirs("plots", exist_ok=True)  # Create output directory

plots = create_scenario_plots(
    baseline.prevalence,
    output_dir="plots/",
    plot_type="individual",  # Just this one scenario
)

plt.show()

---

## 📊 Part 4: Comparing Multiple Scenarios (~25 min)

Now let's compare different intervention strategies.

In [ ]:
# Compare 11 different intervention scenarios
# All start from the same baseline but differ in future interventions

res = run_minter_scenarios(
    scenario_tag = [
        "no_intervention",    # 1. Status quo
        "irs_only",           # 2. Add IRS
        "lsm_only",           # 3. Add LSM
        "py_only_only",       # 4. Standard nets
        "py_only_with_lsm",   # 5. Standard nets + LSM
        "py_pbo_only",        # 6. PBO nets
        "py_pbo_with_lsm",    # 7. PBO nets + LSM
        "py_pyrrole_only",    # 8. Pyrrole nets
        "py_pyrrole_with_lsm",# 9. Pyrrole nets + LSM
        "py_ppf_only",        # 10. PPF (IG2) nets
        "py_ppf_with_lsm",    # 11. PPF nets + LSM
    ],
    
    # Current situation (same for all scenarios)
    res_use    = [0.2] * 11,
    py_only    = [0.3] * 11,
    py_pbo     = [0.2] * 11,
    py_pyrrole = [0.1] * 11,
    py_ppf     = [0.05] * 11,
    prev       = [0.55] * 11,
    Q0         = [0.92] * 11,
    phi        = [0.85] * 11,
    season     = [0] * 11,
    irs        = [0.4] * 11,
    
    # Future interventions (differ by scenario)
    itn_future = [
        0.00,  # no_intervention
        0.00,  # irs_only
        0.00,  # lsm_only
        0.45,  # py_only_only
        0.45,  # py_only_with_lsm
        0.45,  # py_pbo_only
        0.45,  # py_pbo_with_lsm
        0.45,  # py_pyrrole_only
        0.45,  # py_pyrrole_with_lsm
        0.45,  # py_ppf_only
        0.45,  # py_ppf_with_lsm
    ],
    
    net_type_future = [
        None,         # no_intervention
        None,         # irs_only
        None,         # lsm_only
        "py_only",    # py_only_only
        "py_only",    # py_only_with_lsm
        "py_pbo",     # py_pbo_only
        "py_pbo",     # py_pbo_with_lsm
        "py_pyrrole", # py_pyrrole_only
        "py_pyrrole", # py_pyrrole_with_lsm
        "py_ppf",     # py_ppf_only
        "py_ppf",     # py_ppf_with_lsm
    ],
    
    irs_future = [
        0.0,   # no_intervention
        0.5,   # irs_only (50% IRS)
        0.0,   # lsm_only
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  # rest
    ],
    
    routine = [
        0,  # no_intervention
        0,  # irs_only
        0,  # lsm_only
        1, 1, 1, 1, 1, 1, 1, 1,  # All ITN scenarios have routine
    ],
    
    lsm = [
        0.0,  # no_intervention
        0.0,  # irs_only
        0.2,  # lsm_only (20% LSM)
        0.0,  # py_only_only
        0.2,  # py_only_with_lsm
        0.0,  # py_pbo_only
        0.2,  # py_pbo_with_lsm
        0.0,  # py_pyrrole_only
        0.2,  # py_pyrrole_with_lsm
        0.0,  # py_ppf_only
        0.2,  # py_ppf_with_lsm
    ],
)

print(f"✅ Ran {len(res.scenario_meta)} scenarios!")

In [ ]:
# Create combined comparison plot
plots = create_scenario_plots(
    res.prevalence,
    output_dir="plots/",
    plot_type="combined",
)

plt.show()

In [ ]:
# Extract key metrics for comparison
# Get prevalence at year 4 (end of simulation)

summary = res.prevalence.groupby('scenario_tag').apply(
    lambda x: pd.Series({
        'initial_prev': x['prevalence'].iloc[0],
        'final_prev': x['prevalence'].iloc[-1],
        'min_prev': x['prevalence'].min(),
        'reduction': x['prevalence'].iloc[0] - x['prevalence'].iloc[-1],
    })
).round(3)

print("\n📊 Scenario Summary:")
print(summary.sort_values('reduction', ascending=False))

---

## 🔁 Part 5: Batch Processing (~25 min)

For exploring parameter space systematically, you'll want to run many scenarios at once.

### Method 1: Parameter Grids

Explore how outcomes change across different parameter values.

In [ ]:
# Create a grid exploring resistance levels and net coverage
from itertools import product

# Define parameter values to explore
resistance_levels = [0.1, 0.3, 0.5, 0.7, 0.9]  # 5 levels
itn_coverages = [0.2, 0.4, 0.6, 0.8]           # 4 levels

# Create all combinations
combinations = list(product(resistance_levels, itn_coverages))
n_scenarios = len(combinations)

print(f"Total scenarios to run: {n_scenarios}")
print(f"First few combinations: {combinations[:5]}")

In [ ]:
# Unpack the grid into lists for each parameter
res_values = [c[0] for c in combinations]
itn_values = [c[1] for c in combinations]
tags = [f"res{c[0]}_itn{c[1]}" for c in combinations]

# Run all scenarios in one call (efficient!)
grid_results = run_minter_scenarios(
    scenario_tag   = tags,
    res_use        = res_values,
    py_only        = [0.0] * n_scenarios,  # Using PBO nets for this grid
    py_pbo         = itn_values,           # Variable PBO coverage
    py_pyrrole     = [0.0] * n_scenarios,
    py_ppf         = [0.0] * n_scenarios,
    prev           = [0.55] * n_scenarios,
    Q0             = [0.92] * n_scenarios,
    phi            = [0.85] * n_scenarios,
    season         = [0] * n_scenarios,
    routine        = [1] * n_scenarios,
    irs            = [0.0] * n_scenarios,
    irs_future     = [0.0] * n_scenarios,
    lsm            = [0.0] * n_scenarios,
    itn_future     = itn_values,           # Same as current for continuity
    net_type_future= ["py_pbo"] * n_scenarios,
)

print(f"✅ Completed {n_scenarios} scenarios!")

In [ ]:
# Create a heatmap of final prevalence

# Extract final prevalence for each scenario
final_prev = grid_results.prevalence.groupby('scenario_tag')['prevalence'].last()

# Reshape into grid
heatmap_data = np.zeros((len(resistance_levels), len(itn_coverages)))
for i, res in enumerate(resistance_levels):
    for j, itn in enumerate(itn_coverages):
        tag = f"res{res}_itn{itn}"
        heatmap_data[i, j] = final_prev[tag]

# Plot heatmap
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(heatmap_data, cmap='RdYlGn_r', aspect='auto')

ax.set_xticks(range(len(itn_coverages)))
ax.set_xticklabels([f"{x:.0%}" for x in itn_coverages])
ax.set_yticks(range(len(resistance_levels)))
ax.set_yticklabels([f"{x:.0%}" for x in resistance_levels])

ax.set_xlabel('ITN Coverage', fontsize=12)
ax.set_ylabel('Resistance Level', fontsize=12)
ax.set_title('Final Prevalence by Resistance & Coverage', fontsize=14)

# Add value annotations
for i in range(len(resistance_levels)):
    for j in range(len(itn_coverages)):
        text = ax.text(j, i, f"{heatmap_data[i, j]:.1%}",
                       ha="center", va="center", color="black", fontsize=10)

plt.colorbar(im, label='Final Prevalence')
plt.tight_layout()
plt.savefig('plots/resistance_coverage_heatmap.png', dpi=150)
plt.show()

### Method 2: Reading from a CSV/Excel file

For complex scenario sets, prepare your parameters in a spreadsheet.

In [ ]:
# Example: Create a template CSV
template = pd.DataFrame({
    'scenario_tag': ['scenario_1', 'scenario_2', 'scenario_3'],
    'res_use': [0.1, 0.3, 0.5],
    'py_only': [0.3, 0.3, 0.3],
    'py_pbo': [0.2, 0.2, 0.2],
    'py_pyrrole': [0.1, 0.1, 0.1],
    'py_ppf': [0.05, 0.05, 0.05],
    'prev': [0.55, 0.55, 0.55],
    'Q0': [0.92, 0.92, 0.92],
    'phi': [0.85, 0.85, 0.85],
    'season': [0, 0, 0],
    'routine': [1, 1, 1],
    'irs': [0.4, 0.4, 0.4],
    'irs_future': [0.0, 0.0, 0.0],
    'lsm': [0.0, 0.1, 0.2],
    'itn_future': [0.45, 0.45, 0.45],
    'net_type_future': ['py_pbo', 'py_pbo', 'py_pbo'],
})

template.to_csv('scenario_template.csv', index=False)
print("Template saved to scenario_template.csv")
print(template)

In [ ]:
# To upload your own CSV:
# Uncomment the next two lines, then upload your file

# from google.colab import files
# uploaded = files.upload()

# Read the CSV
params = pd.read_csv('scenario_template.csv')

# Convert None strings to actual None
params['net_type_future'] = params['net_type_future'].replace('None', None)

# Run from DataFrame
csv_results = run_minter_scenarios(
    scenario_tag    = params['scenario_tag'].tolist(),
    res_use         = params['res_use'].tolist(),
    py_only         = params['py_only'].tolist(),
    py_pbo          = params['py_pbo'].tolist(),
    py_pyrrole      = params['py_pyrrole'].tolist(),
    py_ppf          = params['py_ppf'].tolist(),
    prev            = params['prev'].tolist(),
    Q0              = params['Q0'].tolist(),
    phi             = params['phi'].tolist(),
    season          = params['season'].tolist(),
    routine         = params['routine'].tolist(),
    irs             = params['irs'].tolist(),
    irs_future      = params['irs_future'].tolist(),
    lsm             = params['lsm'].tolist(),
    itn_future      = params['itn_future'].tolist(),
    net_type_future = params['net_type_future'].tolist(),
)

print(f"✅ Ran {len(params)} scenarios from CSV!")

### Method 3: Helper Function for Cleaner Code

In [ ]:
def run_from_dataframe(df):
    """
    Run scenarios from a pandas DataFrame.
    
    DataFrame should have columns matching the parameter names.
    """
    # Handle None values in net_type_future
    if 'net_type_future' in df.columns:
        net_types = df['net_type_future'].tolist()
        net_types = [None if pd.isna(x) or x == 'None' else x for x in net_types]
    else:
        net_types = [None] * len(df)
    
    return run_minter_scenarios(
        scenario_tag    = df.get('scenario_tag', [f'S{i}' for i in range(len(df))]).tolist(),
        res_use         = df['res_use'].tolist(),
        py_only         = df['py_only'].tolist(),
        py_pbo          = df['py_pbo'].tolist(),
        py_pyrrole      = df['py_pyrrole'].tolist(),
        py_ppf          = df['py_ppf'].tolist(),
        prev            = df['prev'].tolist(),
        Q0              = df['Q0'].tolist(),
        phi             = df['phi'].tolist(),
        season          = df['season'].tolist(),
        routine         = df['routine'].tolist(),
        irs             = df['irs'].tolist(),
        irs_future      = df['irs_future'].tolist(),
        lsm             = df['lsm'].tolist(),
        itn_future      = df.get('itn_future', [0.0] * len(df)).tolist(),
        net_type_future = net_types,
    )

# Use it like this:
# results = run_from_dataframe(my_params_df)
print("Helper function defined! Use: run_from_dataframe(your_dataframe)")

---

## 💾 Part 6: Saving & Exporting Results (~15 min)

In [ ]:
# Save results to CSV
res.prevalence.to_csv('prevalence_results.csv', index=False)
res.cases.to_csv('cases_results.csv', index=False)

print("✅ Saved to CSV files")

In [ ]:
# Download files to your computer
from google.colab import files

# Download individual files
files.download('prevalence_results.csv')
files.download('cases_results.csv')

In [ ]:
# Download all plots as a ZIP
!zip -r plots.zip plots/
files.download('plots.zip')

In [ ]:
# Save to Excel with multiple sheets
with pd.ExcelWriter('minter_results.xlsx') as writer:
    res.prevalence.to_excel(writer, sheet_name='Prevalence', index=False)
    res.cases.to_excel(writer, sheet_name='Cases', index=False)
    res.scenario_meta.to_excel(writer, sheet_name='Metadata', index=False)

files.download('minter_results.xlsx')
print("✅ Downloaded Excel file")

### Saving to Google Drive (persistent storage)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Save to Drive
import shutil

# Create a folder in your Drive
output_folder = '/content/drive/MyDrive/MINT_Results'
os.makedirs(output_folder, exist_ok=True)

# Copy files
shutil.copy('prevalence_results.csv', output_folder)
shutil.copy('cases_results.csv', output_folder)
shutil.copytree('plots', f'{output_folder}/plots', dirs_exist_ok=True)

print(f"✅ Files saved to Google Drive: {output_folder}")

---

## ⚠️ Part 7: Common Issues & Troubleshooting

### Issue 1: List length mismatch

**Error:** `ValueError: All net combination vectors must have the same length`

**Solution:** Make sure all your input lists have the same number of elements.

In [ ]:
# ❌ WRONG - Different lengths
# res_use = [0.2, 0.3]       # 2 elements
# py_only = [0.3, 0.3, 0.3]  # 3 elements - MISMATCH!

# ✅ CORRECT - Same lengths
n = 3  # Number of scenarios
res_use = [0.2] * n  # 3 elements
py_only = [0.3] * n  # 3 elements

print(f"res_use length: {len(res_use)}, py_only length: {len(py_only)}")

### Issue 2: Invalid net type

**Error:** `ValueError: Unknown net_type_future`

**Solution:** Use only valid net types: `py_only`, `py_pbo`, `py_pyrrole`, `py_ppf`, or `None`

In [ ]:
# ❌ WRONG
# net_type_future = ["pbo", "standard", "IG2"]  # Invalid names

# ✅ CORRECT
net_type_future = ["py_pbo", "py_only", "py_ppf"]  # Valid names

print("Valid net types: py_only, py_pbo, py_pyrrole, py_ppf, None")

### Issue 3: EIR out of range

**Warning:** `eir_valid: False`

**Explanation:** The predicted EIR is outside the valid range (0.68 - 350). Results may be unreliable.

In [ ]:
# Check which scenarios have invalid EIR
invalid = res.scenario_meta[~res.scenario_meta['eir_valid']]

if len(invalid) > 0:
    print("⚠️ These scenarios have invalid EIR predictions:")
    print(invalid)
else:
    print("✅ All scenarios have valid EIR predictions")

### Issue 4: Memory errors with large batches

**Solution:** Run in smaller batches.

In [ ]:
def run_in_batches(params_df, batch_size=50):
    """
    Run large scenario sets in batches to avoid memory issues.
    """
    all_prevalence = []
    all_cases = []
    
    n_batches = (len(params_df) + batch_size - 1) // batch_size
    
    for i in range(n_batches):
        start = i * batch_size
        end = min((i + 1) * batch_size, len(params_df))
        batch = params_df.iloc[start:end]
        
        print(f"Running batch {i+1}/{n_batches} (scenarios {start+1}-{end})...")
        
        results = run_from_dataframe(batch)
        
        all_prevalence.append(results.prevalence)
        all_cases.append(results.cases)
    
    # Combine all batches
    combined_prev = pd.concat(all_prevalence, ignore_index=True)
    combined_cases = pd.concat(all_cases, ignore_index=True)
    
    return combined_prev, combined_cases

print("Batch function defined! Use: run_in_batches(your_large_df, batch_size=50)")

---

## 🎓 Part 8: Exercise - Design Your Own Scenarios

Try creating scenarios that represent your region of interest!

In [ ]:
# YOUR TURN!
# Modify these parameters to match your area of interest

my_scenarios = run_minter_scenarios(
    scenario_tag   = ["my_baseline", "my_intervention"],
    
    # Current situation
    res_use        = [0.3, 0.3],   # Resistance level in your area
    py_only        = [0.4, 0.4],   # Current net mix
    py_pbo         = [0.0, 0.0],
    py_pyrrole     = [0.0, 0.0],
    py_ppf         = [0.0, 0.0],
    
    # Epidemiology
    prev           = [0.40, 0.40],  # Your baseline prevalence
    Q0             = [0.90, 0.90],
    phi            = [0.80, 0.80],
    season         = [1, 1],        # Seasonal? (0=no, 1=yes)
    routine        = [1, 1],
    
    # Current interventions
    irs            = [0.0, 0.0],
    
    # Future interventions
    itn_future     = [0.0, 0.60],   # Increase to 60%
    net_type_future= [None, "py_pbo"],  # Switch to PBO nets
    irs_future     = [0.0, 0.0],
    lsm            = [0.0, 0.0],
)

# Plot your results
plots = create_scenario_plots(
    my_scenarios.prevalence,
    plot_type="combined",
)
plt.show()

# Print summary
print("\n📊 Your Results:")
for scenario in my_scenarios.prevalence['scenario_tag'].unique():
    data = my_scenarios.prevalence[my_scenarios.prevalence['scenario_tag'] == scenario]
    print(f"{scenario}: Initial {data['prevalence'].iloc[0]:.1%} → Final {data['prevalence'].iloc[-1]:.1%}")

---

## 📚 Quick Reference Card

### Essential Imports
```python
from minte import run_minter_scenarios, create_scenario_plots
import pandas as pd
```

### Basic Usage
```python
results = run_minter_scenarios(
    scenario_tag = ["name1", "name2"],
    res_use      = [0.2, 0.2],
    # ... other params ...
)

# Access outputs
results.prevalence  # Prevalence DataFrame
results.cases       # Cases DataFrame
results.eir_valid   # Boolean
```

### Plotting
```python
create_scenario_plots(results.prevalence, output_dir="plots/")
```

### Export
```python
results.prevalence.to_csv("output.csv")
from google.colab import files
files.download("output.csv")
```

### Valid Net Types
- `"py_only"` - Standard pyrethroid nets
- `"py_pbo"` - PBO synergist nets
- `"py_pyrrole"` - Pyrrole dual-AI nets
- `"py_ppf"` - PPF/IG2 nets
- `None` - No nets

---

## 🙋 Need Help?

- **Documentation:** [Link to your docs]
- **Issues:** [Link to GitHub issues]
- **Email:** [Your contact email]

---

*Workshop materials by [Your Name] | Last updated: [Date]*